In [1]:
#import packages 
import numpy as np
import pandas as pd
import statistics
from scipy import stats

In [2]:
#import, and slice data (2nd sample group first 100 responses from revised survey, not including 1st sample group of 20)
df = pd.read_csv("UMass Sustainability Initiative.csv", sep=",")
df = df.iloc[0:100]
#df.info()

In [11]:
#data cleaning
df.rename(columns = { 'How old are you?':'Age', 'What is your gender?':'Gender', 'What year are you?':'Year', 'What is your major?':'Major',
                     'Do you live on campus?':'On Campus', 'If you do live on campus, where do live?':'Location',
                     'How often do you recycle?':'Frequency', 'Why do you recycle?':'Reasoning',
                     'Are the recycling rules on campus clearly presented?':'Rules', 
                     'If not, if the recycling rules were clearer would you be more likely to recycle?':'Likelihood',
                     'Do you look for the recycle symbol before recycling?':'Symbol', 'What items do you usually recycle?':'Items',
                     'Did the survey overall make you think about your own personal recycling methods and habits?':'Habits',
                     'Whats the best way to reach you for the prize draw? (Email, Phone, etc. )':'Prize'}, inplace = True)
#print(df.columns)
df.fillna('No', inplace=True)
#dict.fromkeys(['Beer','Alcohol','Beverage','Drink'], 'Drink')
#df['Major'].replace({'CIS', 'Computer Science' :'College of Engineering'}, inplace = True)
df['Major'].replace(dict.fromkeys(['CIS', 'Computer Science', 'Computer science', 'Computer science'], 
                                  'College of Engineering'), inplace = True)
df['Major'].replace(dict.fromkeys(['MIS', 'Accounting', 'Marketing'], 'College of Business'), inplace = True)

#df['Major'] = df['Major'].map({'CIS':'College of Engineering'})
df.head(3)

,Timestamp,Age,Gender,Year,Major,On Campus,Location,Frequency,Reasoning,Rules,Likelihood,Symbol,Items,Habits,Prize
0,2020/02/18 3:03:42 PM EST,20,Male,Sophomore,College of Engineering,Yes,Birch,Often,"[Sustainability, Ease of Access]",Yes,Yes,Yes,"[Paper, Cardboard, Plastic Bottles, Glass Bott...",3,No
1,2020/02/18 3:05:49 PM EST,20,Male,Junior,College of Engineering,Yes,Aspen,Always,"[Sustainability, Habit]",No,Yes,No,"[Paper, Cardboard, Plastic Bottles, Glass Bott...",4,jegan2@umassd.edu
2,2020/02/18 3:09:10 PM EST,23,Male,Junior,College of Engineering,No,No,Always,[Habit],Yes,No,No,"[Paper, Cardboard, Plastic Bottles, Glass Bott...",2,No


In [9]:
#view frequency of responses
#df['Reasoning'].value_counts(dropna=False)
df['Major'].value_counts(dropna=False)

College of Engineering                 10
Nursing                                10
College of Business                     7
Computer Engineering                    4
Mechanical Engineering                  3
Psychology                              3
Data Science                            3
Bioengineering                          2
Health and society                      2
Electrical engineering                  2
health and society                      2
Computer engineering                    2
Mechanical engineering                  2
Finance                                 2
Marketing                               2
Electrical engineering                  2
Biology                                 2
Computer Science                        2
Civil Engineering                       1
Computer Engineering                    1
Undeclared                              1
Crime and Justice Studies               1
accounting                              1
Political science                 

In [4]:
#basic data analysis for quantitive data, habits and age distribution
def distribution(df):
    print(f'The mean is {df.mean()} the Standard Erorr of the mean is {stats.sem(df, axis=None, ddof=0)}')
    print(f'The variance is {statistics.variance(df)} the standard deviation is {df.std()}')
    print(f'The min is {df.min()}, Q1 is {np.quantile(df, .25)}, median is {df.median()}, Q3 is {np.quantile(df, .75)}, max is {df.max()}')
    print(f'Skewness: {stats.skew(df)}')
        
distribution(df['Habits'])


The mean is 3.67 the Standard Erorr of the mean is 0.11580587204455565
The variance is 1.3546464646464647 the standard deviation is 1.1638928063384812
The min is 1, Q1 is 3.0, median is 4.0, Q3 is 5.0, max is 5
Skewness: -0.8025214276103186


In [5]:
#Age doesn't work for some reason with the distribution function
#Typer Error: can't multiply sequence by non-int of type 'float' there's another error thats says something about dividing str and ints
#code that doesn't work is commented out below
#if you push a jupyter notebook with errors it will push the error log as well btw, same goes for all output 
df.Age = df.Age.astype(int)
distribution(df['Age'])
#df['Age'].min()
#np.percentile(df['Age'], 25)
#if you check the frequency of values they're all ints (int64) so I'm not sure why there's errors since distr. works for Habits
df['Age'].value_counts(dropna=False)
#df['Age'].unique()
#df['Age'].describe()

The mean is 20.23 the Standard Erorr of the mean is 0.14060938802227968
The variance is 1.997070707070707 the standard deviation is 1.4131775214284672
The min is 18, Q1 is 19.0, median is 20.0, Q3 is 21.0, max is 24
Skewness: 0.5372806957637466


20    28
21    25
19    22
18    10
22     8
23     4
24     3
Name: Age, dtype: int64

In [6]:
#Sorting entries (potential group of interest could be people with low habits, majority of people had high schore for habits)
df.sort_values('Habits', ascending=True)

,Timestamp,Age,Gender,Year,Major,On Campus,Location,Frequency,Reasoning,Rules,Likelihood,Symbol,Items,Habits,Prize
32,2020/02/18 3:39:14 PM EST,19,Male,Sophomore,Mechanical engineering,Yes,Pine Dale,Always,Ease of Access,Yes,Yes,Yes,Paper;Cardboard;Plastic Bottles;Glass Bottles;...,1,cmcnulty@umassd.edu 774-606-4474
21,2020/02/18 3:37:36 PM EST,20,Female,Junior,Nursing,Yes,Ivy,Never,Habit,No,Yes,Yes,Paper;Cardboard;Plastic Bottles,1,7745786433
98,2020/02/18 4:15:27 PM EST,19,Male,Sophomore,Data Science !,Yes,Dells,Often,Habit,I didn't know there were any recycling rules!,No,Yes,Paper;Cardboard;Plastic Bottles;Aluminum Cans,1,sbalkus@umassd.edu
53,2020/02/18 3:48:28 PM EST,18,Female,Freshman,nursing,Yes,Elmwood,Often,Sustainability,Yes,No,Yes,Paper;Cardboard;Plastic Bottles;Plastic Bags;G...,1,No
6,2020/02/18 3:13:42 PM EST,21,Male,Junior,College of Business,No,No,Often,Sustainability,Yes,Yes,Yes,Paper;Cardboard;Plastic Bottles,1,tuddin@umassd.edu
27,2020/02/18 3:38:35 PM EST,23,Male,Senior,Political Science,No,No,Rarely,Ease of Access,No,No,No,Paper,1,774-766-2602
75,2020/02/18 3:51:42 PM EST,24,Male,Sophomore,Electrical engineering,No,No,Always,Habit,Yes,No,Yes,Plastic Bottles,1,No
11,2020/02/18 3:26:08 PM EST,20,Male,Sophomore,Comp sci,Yes,Dells,Always,Sustainability;Ease of Access;Habit,I didn't know there were any recycling rules!,Yes,Yes,Paper;Cardboard;Plastic Bottles;Plastic Bags;G...,1,Tinder
23,2020/02/18 3:37:47 PM EST,20,Male,Junior,Mathematics,Yes,Willow,Always,Sustainability;Habit,Yes,No,Yes,Paper;Cardboard;Plastic Bottles;Glass Bottles;...,2,"email, dschiek@umassd"
51,2020/02/18 3:47:25 PM EST,20,Female,Freshman,Electrical engineering,No,No,Always,Habit,Yes,No,Yes,Paper;Cardboard;Plastic Bottles;Plastic Bags;G...,2,vsilvia@umassd.edu


In [7]:
df['Reasoning'] = df['Reasoning'].str.split(";", expand = False)
df['Items'] = df['Items'].str.split(";", expand = False)
#df['Why do you recycle?']
type(df)

pandas.core.frame.DataFrame

In [8]:
type(df['Reasoning'][0][0])
df['Reasoning']

0            [Sustainability, Ease of Access]
1                     [Sustainability, Habit]
2                                     [Habit]
3     [Sustainability, Ease of Access, Habit]
4                            [Sustainability]
5                            [Ease of Access]
6                            [Sustainability]
7                     [Sustainability, Habit]
8            [Sustainability, Ease of Access]
9                     [Ease of Access, Habit]
10                                    [Habit]
11    [Sustainability, Ease of Access, Habit]
12                                    [Habit]
13                                    [Habit]
14                                    [Habit]
15                           [Ease of Access]
16                    [Sustainability, Habit]
17                    [Sustainability, Habit]
18                                    [Habit]
19                           [Sustainability]
20                                    [Habit]
21                                